In [2]:
# Load necessary packages
from osgeo import gdal, osr
import numpy as np
import pandas as pd
import os

In [14]:
# Bounding box long and lat for Eastern US and Canada
min_long = -87.75
min_lat = 25.0
max_long = -52.75
max_lat = 62.5

# Divide bounidng box into three equal regions which roughly equate 
# to southern US, Northern US, and Canada
split_lat = (max_lat - min_lat ) / 3

print(split_lat)
print("Canada region bounded by 62.5 lattitude and " + str(62.5 - split_lat) + " latitude")
print("Northern US region bounded by "+ str(62.5 - split_lat) + " latitude and " +  str(62.5 - split_lat*2) + " latitude")
print("Southern US region bounded by "+ str(62.5 - split_lat*2) + " latitude and " +  str(62.5 - split_lat*3) + " latitude")

12.5
Canada region bounded by 62.5 lattitude and 50.0 latitude
Northern US region bounded by 50.0 latitude and 37.5 latitude
Southern US region bounded by 37.5 latitude and 25.0 latitude


In [15]:
# First Grid in top left corner the left longitude and top lattitude bounding
grid_long_start = -180
grid_lat_start = 90

# Find index in 720 x 1440 array in which the min lat and long are
# Using adjusted formula from User Guide to calculate left (long) and bottom (lat)
# of the Grid
grid_min_long_index = int((min_long - grid_long_start) / 0.25)
grid_min_lat_index = int((min_lat - grid_lat_start) / -0.25)
print("Min long grid index {}".format(grid_min_long_index))
print("Min lat grid index {}".format(grid_min_lat_index))

# Find index for max lat and long bbox
grid_max_long_index = int((max_long - grid_long_start) / 0.25 - 1) # Shift left one grid 
grid_max_lat_index = int((max_lat - grid_lat_start) / -0.25 - 1) # Shift down one grid
print("Max long grid index {}".format(grid_max_long_index))
print("Max lat grid index {}".format(grid_max_lat_index))

# Sanity check the indices
print(min_long == -180 + grid_min_long_index * 0.25)
print(min_lat == 90 - grid_min_lat_index * 0.25)
print(max_long == -180 + grid_max_long_index * 0.25 + 0.25) # Shift left one grid 
print(max_lat == 90 - grid_max_lat_index * 0.25 - 0.25) # Shift down one grid

Min long grid index 369
Min lat grid index 260
Max long grid index 508
Max lat grid index 109
True
True
True
True


In [16]:
def get_sds(filename):
    '''Opens HDF file in GDAL and get data layers in SDS list'''
    hdf_handle = gdal.Open(filename)
    if hdf_handle is None:
        print("Something is wrong with {}".format(filename))
        return None
    sds_list = hdf_handle.GetSubDatasets()
    return sds_list

In [17]:
def get_burned_area(input_sds, min_long_idx, max_long_idx, f_name):
    '''Returns a pandas dataframe with 3 rows of data representing the burned area 
    of input file and bounding longitude and lattitudes, broken out by region: Southeastern US, 
    Northeastern US, and Eastern Canada. Note since the grid starts at -180 longitude and 90 latitude, 
    longitude increases as the index increases, but lattitude decreases as the index increases, 
    so the minimum lattitude bounding box has a larger index.'''
    
    if input_sds is None:
        return pd.DataFrame({'FileName' : [], 'StartDate' :  [], 'EndDate': [],
                            'BurnedArea': [], 'Units' : [], 'ScaleFactor' : []})
    # Extract burned area data
    burn_area_handle = gdal.Open(input_sds[0][0])
    burn_area_data = burn_area_handle.ReadAsArray()
    burn_area_start = burn_area_handle.GetMetadata()['StartDate']
    burn_area_end = burn_area_handle.GetMetadata()['EndDate']
    burn_area_units = burn_area_handle.GetMetadata()['units']
    burn_area_scale = float(burn_area_handle.GetMetadata()['scale_factor'])
    
    # Create empty dataframe to append to 
    burn_df = pd.DataFrame({'FileName' : [], 'StartDate' :  [], 'EndDate': [],
                            'BurnedArea': [], 'Region':[], 'Units' : [], 'ScaleFactor' : []})
    
    # Create list with indices splitting into three regions
    index_split = [(261, 210), (210, 159), (159, 109)] 
    region_list = ["Southeastern US", "Northeastern US", "Eastern Canada", ]
        
    # Counter to ensure accurate grid is selected
    grid_counter = 0

    # Loop through 3 regions and calculate burned area data
    for i in range(3):
        region_burn_data = burn_area_data[index_split[i][1]:index_split[i][0], min_long_idx:max_long_idx + 1]
        grid_counter += region_burn_data.size
        total_burned = np.sum(burn_area_scale * region_burn_data)
        add_df = pd.DataFrame({'FileName' : f_name, 'StartDate' :  burn_area_start, 'EndDate': burn_area_end,
                            'BurnedArea': total_burned, 'Region': region_list[i], 'Units' : burn_area_units, 
                            'ScaleFactor' : burn_area_scale}, index = [0])
        burn_df = pd.concat([burn_df, add_df])

    if grid_counter != 21280:
        print(grid_counter)
        print('For {} there is something wrong with the subsetted grid size.'.format(f_name))

    return burn_df

In [18]:
def get_burned_area_coords(input_sds, min_long_idx, max_long_idx, f_name):
    '''Returns a pandas dataframe that is subsetted based on the bounding box inputs 
    where a single row represents the burened area for 0.25 degree grid from the MODSI 
    Burned Area product'''

    if input_sds is None:
        return pd.DataFrame({'FileName' : [], 'StartDate' :  [], 'EndDate': [],
                            'BurnedArea': [], 'Latitude' : [], 'Longitude' : [], 
                            'Units' : [], 'ScaleFactor' : []})
    # Extract burned area data
    burn_area_handle = gdal.Open(input_sds[0][0])
    burn_area_data = burn_area_handle.ReadAsArray()
    burn_area_start = burn_area_handle.GetMetadata()['StartDate']
    burn_area_end = burn_area_handle.GetMetadata()['EndDate']
    burn_area_units = burn_area_handle.GetMetadata()['units']
    burn_area_scale = float(burn_area_handle.GetMetadata()['scale_factor'])
    
    # Get lat long coordinates for center of each grid
    lat_coordinates = [ (90 - 0.25/2) - 0.25 * i for i in range(burn_area_data.shape[0])]
    long_coordinates = [(-180 + 0.25 / 2) + 0.25* i for i in range(burn_area_data.shape[1])]

    # Create array with [lat, long] coordinates
    center_coord = np.array([[lat, long] for lat in lat_coordinates for long in long_coordinates])

    # Reshape arraey to 720 x 1440 with [lat, long] in each position
    # and subset to bounding box
    lat = center_coord.reshape((720, 1440, 2))[109:261, min_long_idx:max_long_idx + 1, 0]
    long = center_coord.reshape((720, 1440, 2))[109:261, min_long_idx:max_long_idx + 1, 1]

    # Subset burned area array to bounding box
    burn_area_filter = burn_area_data[109:261, min_long_idx:max_long_idx + 1]

    # Apply Scale Factor
    burn_area_filter = burn_area_filter * burn_area_scale

    # Get array size to fill out dataframe
    arr_size = burn_area_filter.size

    # Create empty dataframe to append to 
    burn_df = pd.DataFrame({'FileName' : np.repeat(f_name, arr_size), 'StartDate' :  np.repeat(burn_area_start, arr_size), 
                            'EndDate': np.repeat(burn_area_end, arr_size), 'BurnedArea': burn_area_filter.flatten(), 
                            'Latitude' : lat.flatten(), 'Longitude' : long.flatten()})

    if arr_size != 21280:
        print('For {} there is something wrong with the subsetted grid size.'.format(f_name))

    return burn_df

In [19]:
# Create empty dataframe to append to while looping through files
# This will store the total burned area of the bounding box
total_burn_df = pd.DataFrame({'FileName' : [], 'StartDate' :  [], 'EndDate': [],
                            'BurnedArea': [], 'Units' : [], 'ScaleFactor' : []})

# Create empty dataframe to store the grid coordiantes burned area
coord_burn_df = pd.DataFrame({'FileName' : [], 'StartDate' :  [], 'EndDate': [],
                            'BurnedArea': [], 'Latitude' : [], 'Longitude' : []})

# Set folder directory where CMG files are located
folder_dir = 'MODIS CMG Burned Area'

# Loop through each file to calculate burned area and append to output dataframe
for f in os.listdir(folder_dir):
    print("Calculating burned area for {}".format(f))
    file_path = folder_dir + "\\" + f
    sds = get_sds(file_path)

    # Total burned area
    total_append_df = get_burned_area(sds, grid_min_long_index, grid_max_long_index, f)
    total_burn_df = pd.concat([total_burn_df, total_append_df])

    # Burned area by grid
    coord_append_df = get_burned_area_coords(sds, grid_min_long_index, grid_max_long_index, f)
    coord_burn_df = pd.concat([coord_burn_df, coord_append_df])

Calculating burned area for MCD64CMQ.A2000306.006.2018149224217.hdf
Calculating burned area for MCD64CMQ.A2000336.006.2018149224618.hdf
Calculating burned area for MCD64CMQ.A2001001.006.2018149224951.hdf
Calculating burned area for MCD64CMQ.A2001032.006.2018149225319.hdf
Calculating burned area for MCD64CMQ.A2001060.006.2018149225703.hdf
Calculating burned area for MCD64CMQ.A2001091.006.2018149230031.hdf
Calculating burned area for MCD64CMQ.A2001121.006.2018149230404.hdf
Calculating burned area for MCD64CMQ.A2001152.006.2018149230736.hdf
Calculating burned area for MCD64CMQ.A2001182.006.2018149231111.hdf
Calculating burned area for MCD64CMQ.A2001213.006.2018149231448.hdf
Calculating burned area for MCD64CMQ.A2001244.006.2018149231839.hdf
Calculating burned area for MCD64CMQ.A2001274.006.2018149232217.hdf
Calculating burned area for MCD64CMQ.A2001305.006.2018149232553.hdf
Calculating burned area for MCD64CMQ.A2001335.006.2018149232949.hdf
Calculating burned area for MCD64CMQ.A2002001.00

In [25]:
# Export to csv
final_burn_df.to_csv("region_breakout_monthly_burned.csv", index = False)

In [20]:
#Export to parquet
coord_burn_df.to_parquet("burned_area_coordinates.parquet")